# Coding Block 4 - Automated model and hyperparameter tuning with AutoGluon

### Load the packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# AutoML
from autogluon.tabular import TabularPredictor
'''
...
'''

'\n...\n'

### Read the dataset 
You can also compare processed and non-processed data. The autogluon library will do some preprocessing as well.

In [2]:
diab_cleaned=pd.read_csv('diabetes_cleaned.csv')
diab_cleaned.drop(columns=['outlier_z_score', 'outlier_Tukey'],
                   errors='ignore', inplace=True)

In [3]:
diab_cleaned.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.716146,72.522135,29.438802,155.583333,32.410677,0.471876,33.240885,0.348958
std,3.369578,30.649214,12.437674,10.488407,118.777435,6.930788,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,77.000000,27.400000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.500000,32.250000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.250000,194.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Use the Autogluon library
Use the library autogluon for automated hyperparametertuning and model benchmarking. The fit function of the TabularPredictor object allows for setting the option: <br>
<i>presets='medium_quality'</i> <br>
which limits the depths of hyperparameter optimization

In [ ]:
# Define target variable
target = 'Outcome'

# Define time limit for model training (in seconds)
TIME_LIMIT = 600  # 10 minutes

# Split data into train/test sets
train_data, test_data = train_test_split(diab_cleaned, test_size=0.2, 

# Train AutoGluon model with the desired adjustments
predictor = TabularPredictor(label=target, eval_metric='roc_auc').fit(
    train_data,
    presets='best_quality',  # Using the best quality preset for higher accuracy
    time_limit=TIME_LIMIT,  # You can set a time limit to control how long the training runs (in seconds)
    num_bag_folds=5,  # Enable ensembling by using bagging with 8 folds
    num_stack_levels=1,  # Enable stacking to further improve model performance
    ag_args_fit={'use_gpu': False},  # Set to True to use GPU for model training
    verbosity=0  # Set verbosity level to suppress too much logging output (0: most silent, 4: most detailed)
)

# Evaluate model performance on test data
print("\n--- Model Performance on Test Data ---")
test_performance = predictor.evaluate(test_data, detailed_report=True)
print(f"Test ROC-AUC: {test_performance['roc_auc']:.4f}")
print(f"Test Accuracy: {test_performance['accuracy']:.4f}")
print(f"Test F1 Score: {test_performance['f1']:.4f}")


### Show the leaderboard
The resulting "predictor" object of the TabularPredictor function from Autogluon has a function "leaderboard"

In [ ]:
# Get model leaderboard
print("\n--- Model Leaderboard ---")
leaderboard = predictor.leaderboard(test_data)
print(leaderboard)

### Show the feature importance table
The resulting "predictor" object of the TabularPredictor function from Autogluon also has a function "feature_importance"

In [ ]:
# Feature importance
print("\n--- Feature Importance ---")
try:
    importance = predictor.feature_importance(test_data)
    print(importance)
except:
    print("Feature importance calculation not supported for the best model")

# Make predictions on test data
y_pred = predictor.predict(test_data)
y_pred_proba = predictor.predict_proba(test_data)

# Get confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print("\n--- Confusion Matrix ---")
print(confusion_matrix(test_data[target], y_pred))

# Get classification report
print("\n--- Classification Report ---")
print(classification_report(test_data[target], y_pred))